In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import kagglehub

In [15]:
path = kagglehub.dataset_download("demartlectus/wildberries-search-queries-from-wildhack-2021")
file_path = f"{path}/query_popularity.csv"
data = pd.read_csv(file_path)
print(data.head())

                    query  query_popularity
0                 ноутбук                10
1  куртка женская осенняя                10
2         ботинки женские                10
3              видеокарта                10
4  пальто женское осеннее                10


In [22]:
# Создание целевой переменной (категории популярнсоти)
def categorize_popularity(popularity):
    if popularity >= 8:
        return "Популярный запрос"
    elif popularity >= 5:
        return "Средне популярный запрос"
    else:
        return "Не популярный запрос"

data['popularity_category'] = data['query_popularity'].apply(categorize_popularity)
data['query'] = data['query'].fillna("")

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data['query'], data['popularity_category'], test_size=0.2, random_state=50)

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение классификатора
classifier = MultinomialNB()
classifier.fit(X_train_vec, y_train)

# Предсказание на тестовых данных
y_pred = classifier.predict(X_test_vec)

# Оценка качества модели
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Classification Report:
                          precision    recall  f1-score   support

    Не популярный запрос       0.55      0.82      0.66     27035
       Популярный запрос       0.62      0.51      0.56     20243
Средне популярный запрос       0.40      0.20      0.27     20120

                accuracy                           0.54     67398
               macro avg       0.52      0.51      0.49     67398
            weighted avg       0.52      0.54      0.51     67398

Accuracy: 0.5421674233656785


In [23]:
external_data = pd.read_csv(file_path)
external_data['popularity_category'] = external_data['query_popularity'].apply(categorize_popularity)
external_data['query'] = external_data['query'].fillna("")

# Векторизация текста
X_external_vec = vectorizer.transform(external_data['query'])
y_external_pred = classifier.predict(X_external_vec)

# Оценка качества модели на внешних данных
print("External Data Classification Report:")
print(classification_report(external_data['popularity_category'], y_external_pred))
print(f"External Data Accuracy: {accuracy_score(external_data['popularity_category'], y_external_pred)}")

External Data Classification Report:
                          precision    recall  f1-score   support

    Не популярный запрос       0.55      0.83      0.66    134723
       Популярный запрос       0.62      0.52      0.57    101102
Средне популярный запрос       0.45      0.22      0.30    101163

                accuracy                           0.55    336988
               macro avg       0.54      0.52      0.51    336988
            weighted avg       0.54      0.55      0.53    336988

External Data Accuracy: 0.5549633814853941


In [24]:
# Функция для предсказания категории запроса
def predict_query_category(query, vectorizer, classifier):
    # Очистка текста (заполнение пропущенных значений, если есть)
    query = query if isinstance(query, str) else ""

    # Преобразование текста в числовой формат
    query_vec = vectorizer.transform([query])

    # Предсказание категории
    predicted_category = classifier.predict(query_vec)

    return predicted_category[0]

# Пример использования
user_query = input("Введите ваш запрос: ")  # Пользователь вводит запрос
predicted_category = predict_query_category(user_query, vectorizer, classifier)
print(f"Запрос '{user_query}' относится к категории: {predicted_category}")

Введите ваш запрос: шуба из овечьей шерсти
Запрос 'шуба из овечьей шерсти' относится к категории: Популярный запрос
